In [20]:
# install the pycuda library
!pip install pycuda

import pycuda.autoinit
import pycuda.driver as cuda

# check available GPUs
print("%d device(s) found." % cuda.Device.count())

# save device information
dev = cuda.Device(0)
print("Device: %s", dev.name()) # device name
print(" Compute Capability: %d.%d" % dev.compute_capability()) # device compute capability
print(" Total Memory: %s KB" % (dev.total_memory()//(1024))) # device memory

# all the device attributes in sorted order
atts = [(str(att), value)
 for att, value in dev.get_attributes().items()]
atts.sort()
 
for att, value in atts:
 print(" %s: %s" % (att, value))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
1 device(s) found.
Device: %s Tesla T4
 Compute Capability: 7.5
 Total Memory: 15464256 KB
 ASYNC_ENGINE_COUNT: 3
 CAN_MAP_HOST_MEMORY: 1
 CAN_USE_HOST_POINTER_FOR_REGISTERED_MEM: 1
 CLOCK_RATE: 1590000
 COMPUTE_CAPABILITY_MAJOR: 7
 COMPUTE_CAPABILITY_MINOR: 5
 COMPUTE_MODE: DEFAULT
 COMPUTE_PREEMPTION_SUPPORTED: 1
 CONCURRENT_KERNELS: 1
 CONCURRENT_MANAGED_ACCESS: 1
 DIRECT_MANAGED_MEM_ACCESS_FROM_HOST: 0
 ECC_ENABLED: 1
 GENERIC_COMPRESSION_SUPPORTED: 0
 GLOBAL_L1_CACHE_SUPPORTED: 1
 GLOBAL_MEMORY_BUS_WIDTH: 256
 GPU_OVERLAP: 1
 HANDLE_TYPE_POSIX_FILE_DESCRIPTOR_SUPPORTED: 1
 HANDLE_TYPE_WIN32_HANDLE_SUPPORTED: 0
 HANDLE_TYPE_WIN32_KMT_HANDLE_SUPPORTED: 0
 HOST_NATIVE_ATOMIC_SUPPORTED: 0
 INTEGRATED: 0
 KERNEL_EXEC_TIMEOUT: 0
 L2_CACHE_SIZE: 4194304
 LOCAL_L1_CACHE_SUPPORTED: 1
 MANAGED_MEMORY: 1
 MAXIMUM_SURFACE1D_LAYERED_LAYERS: 2048
 MAXIMUM_SURFACE1D_LAYERED_WIDTH: 32768
 MAXIMUM_SU

# Matrix Multiplication Naive


In [21]:
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
from pycuda.compiler import SourceModule

BLOCK_SIZE = 16

# Define the matrix multiplication kernel
mod = SourceModule("""
__global__ void matrixMult(float *A, float *B, float *C, int size)
{
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int k;
    if (row < size && col < size)
    {
        float sum = 0.0f;
        for (k = 0; k < size; k++)
        {
            sum += A[row * size + k] * B[k * size + col];
        }
        C[row * size + col] = sum;
    }
}
""")

# Get the kernel function
matrixMult = mod.get_function("matrixMult")

if __name__ == '__main__':
    import sys
    if len(sys.argv) != 2:
        print("Usage: {} size".format(sys.argv[0]))
        exit(1)

    size = 5
    input_size = size * size * np.dtype(np.float32).itemsize
    if size <= 0:
        print("Invalid matrix size: {}".format(size))
        exit(1)

    # Allocate memory for matrices A, B, and C on the host
    A = np.random.rand(size, size).astype(np.float32)
    B = np.random.rand(size, size).astype(np.float32)
    C = np.zeros((size, size), dtype=np.float32)

    # Allocate memory for matrices A, B, and C on the device
    dev_a = cuda.mem_alloc(input_size)
    dev_b = cuda.mem_alloc(input_size)
    dev_c = cuda.mem_alloc(input_size)

    # Copy matrices A and B from host to device
    cuda.memcpy_htod(dev_a, A)
    cuda.memcpy_htod(dev_b, B)

    # Define the grid and block dimensions for the MatrixMultKernel
    dimBlock = (BLOCK_SIZE, BLOCK_SIZE, 1)
    dimGrid = ((size + BLOCK_SIZE - 1) // BLOCK_SIZE, (size + BLOCK_SIZE - 1) // BLOCK_SIZE)

    # Create CUDA events to measure the execution time
    start = cuda.Event()
    stop = cuda.Event()

    # Call the MatrixMultKernel on the device
    start.record()
    matrixMult(dev_a, dev_b, dev_c, np.int32(size), block=dimBlock, grid=dimGrid)
    stop.record()
    stop.synchronize()

    # Copy matrix C from device to host
    cuda.memcpy_dtoh(C, dev_c)

    # Calculate the elapsed time in seconds
    elapsedTime = stop.time_since(start)

    # Print the execution time
    print("Execution time: {} ms".format(elapsedTime))
    
    # Free memory
    del A, B, C
    dev_a.free()
    dev_b.free()
    dev_c.free()
  


Usage: /usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py size
Execution time: 0.0936959981918335 ms


# Matrix Multiplication Tiled


In [26]:
import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule
import numpy as np
import time

BLOCK_SIZE = 16

mod = SourceModule("""
#define TILE_SIZE 16
__global__ void matrixMult(float *A, float *B, float *C, int size)
{
    __shared__ float As[TILE_SIZE][TILE_SIZE];
    __shared__ float Bs[TILE_SIZE][TILE_SIZE];

    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;

    int row = by * TILE_SIZE + ty;
    int col = bx * TILE_SIZE + tx;

    float sum = 0.0f;

    for (int k = 0; k < size / TILE_SIZE; k++)
    {
        // Load tiles into shared memory
        As[ty][tx] = A[row * size + k * TILE_SIZE + tx];
        Bs[ty][tx] = B[(k * TILE_SIZE + ty) * size + col];

        // Synchronize to ensure all tiles are loaded
        __syncthreads();

        // Multiply the tiles and accumulate the result
        for (int i = 0; i < TILE_SIZE; i++)
        {
            sum += As[ty][i] * Bs[i][tx];
        }

        // Synchronize to ensure all tiles are used before overwriting shared memory
        __syncthreads();
    }

    // Write the result to the output matrix
    if (row < size && col < size)
    {
        C[row * size + col] = sum;
    }
}
""")

matrixMult = mod.get_function("matrixMult")

def main(size):

    input_size = size * size * np.dtype(np.float32).itemsize

    if size <= 0:
        print("Invalid matrix size: ", size)
        return

    # Allocate memory for matrices A, B, and C on the host
    A = np.random.rand(size, size).astype(np.float32)
    B = np.random.rand(size, size).astype(np.float32)
    C = np.empty_like(A)

    # Allocate memory for matrices A, B, and C on the device
    dev_a = drv.mem_alloc(input_size)
    dev_b = drv.mem_alloc(input_size)
    dev_c = drv.mem_alloc(input_size)

       # Copy matrices A and B from host to device
    drv.memcpy_htod(dev_a, A)
    drv.memcpy_htod(dev_b, B)

    # Define the grid and block dimensions for the MatrixMultKernel
    dimBlock = (BLOCK_SIZE, BLOCK_SIZE, 1)
    dimGrid = ((size + BLOCK_SIZE - 1) // BLOCK_SIZE, (size + BLOCK_SIZE - 1) // BLOCK_SIZE, 1)

    # Create CUDA events to measure the execution time
    start = drv.Event()
    stop = drv.Event()

    # Call the MatrixMultKernel on the device
    start.record()
    matrixMult(dev_a, dev_b, dev_c, np.int32(size), block=dimBlock, grid=dimGrid)
    stop.record()
    stop.synchronize()

    # Copy matrix C from device to host
    drv.memcpy_dtoh(C, dev_c)

    # Calculate the elapsed time in milliseconds
    elapsed_time = stop.time_since(start)

    # Print the execution time
    print("Execution time: {:.3f} ms".format(elapsed_time))

  

if __name__ == "__main__":
    main(5)


Execution time: 0.119 ms


# Matrix Multiplication Atomics


In [29]:
import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule
import numpy as np
import time

BLOCK_SIZE = 16

mod = SourceModule("""
    #define BLOCK_SIZE 16
    __global__ void matrixMult(float *A, float *B, float *C, int size) {
        __shared__ float tileA[BLOCK_SIZE][BLOCK_SIZE];
        __shared__ float tileB[BLOCK_SIZE][BLOCK_SIZE];

        int row = blockIdx.y * blockDim.y + threadIdx.y;
        int col = blockIdx.x * blockDim.x + threadIdx.x;
        float sum = 0.0f;

        for (int t = 0; t < (size + BLOCK_SIZE - 1) / BLOCK_SIZE; t++) {
            int tiledRow = blockIdx.y * blockDim.y + threadIdx.y;
            int tiledCol = t * blockDim.x + threadIdx.x;

            if (tiledRow < size && tiledCol < size)
                tileA[threadIdx.y][threadIdx.x] = A[tiledRow * size + tiledCol];
            else
                tileA[threadIdx.y][threadIdx.x] = 0.0;

            tiledRow = t * blockDim.y + threadIdx.y;
            tiledCol = blockIdx.x * blockDim.x + threadIdx.x;

            if (tiledRow < size && tiledCol < size)
                tileB[threadIdx.y][threadIdx.x] = B[tiledRow * size + tiledCol];
            else
                tileB[threadIdx.y][threadIdx.x] = 0.0;

            __syncthreads();

            for (int i = 0; i < BLOCK_SIZE; i++) {
                sum += tileA[threadIdx.y][i] * tileB[i][threadIdx.x];
            }

            __syncthreads();
        }

        if (row < size && col < size)
            atomicAdd(&C[row * size + col], sum);
    }
    """)

matrixMult = mod.get_function("matrixMult")

def main(size):

    input_size = size * size * np.dtype(np.float32).itemsize

    if size <= 0:
        print("Invalid matrix size: ", size)
        return

    # Allocate memory for matrices A, B, and C on the host
    A = np.random.rand(size, size).astype(np.float32)
    B = np.random.rand(size, size).astype(np.float32)
    C = np.empty_like(A)

    # Allocate memory for matrices A, B, and C on the device
    dev_a = drv.mem_alloc(input_size)
    dev_b = drv.mem_alloc(input_size)
    dev_c = drv.mem_alloc(input_size)

       # Copy matrices A and B from host to device
    drv.memcpy_htod(dev_a, A)
    drv.memcpy_htod(dev_b, B)

    # Define the grid and block dimensions for the MatrixMultKernel
    dimBlock = (BLOCK_SIZE, BLOCK_SIZE, 1)
    dimGrid = ((size + BLOCK_SIZE - 1) // BLOCK_SIZE, (size + BLOCK_SIZE - 1) // BLOCK_SIZE, 1)

    # Create CUDA events to measure the execution time
    start = drv.Event()
    stop = drv.Event()

    # Call the MatrixMultKernel on the device
    start.record()
    matrixMult(dev_a, dev_b, dev_c, np.int32(size), block=dimBlock, grid=dimGrid)
    stop.record()
    stop.synchronize()

    # Copy matrix C from device to host
    drv.memcpy_dtoh(C, dev_c)

    # Calculate the elapsed time in milliseconds
    elapsed_time = stop.time_since(start)

    # Print the execution time
    print("Execution time: {:.3f} ms".format(elapsed_time))

  

if __name__ == "__main__":
    main(5)


Execution time: 0.109 ms


/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: module in out-of-thread context could not be cleaned up
  globals().clear()
